In [1]:
import openmmtools
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import numpy as np
from importlib import reload

In [2]:
pdb = app.PDBFile('tip3p.pdb')
forcefield = app.ForceField('tip3p.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers, rigidWater=True, 
    ewaldErrorTolerance=0.0005)

integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 
    1.0*unit.femtoseconds)

system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 300*unit.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [3]:
reload(openmmtools.alchemy)
factory = openmmtools.alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False, split_alchemical_forces = True, alchemical_pme_treatment = 'exact')
reference_system = system

alchemical_region_0 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [0,1,2], name='zero')
alchemical_region_1 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [3,4,5], name='one')
#alchemical_region_2 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [18,19,20,21,22,23,24,25,26], alchemical_angles = [32],  name='two')

alchemical_system_in = factory.create_alchemical_system(reference_system, alchemical_regions = [alchemical_region_0, alchemical_region_1])

alchemical_state_zero = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'zero')
alchemical_state_one = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'one')
#alchemical_state_two = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'two')

Using 2 alchemical regions
 [<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f9c03dcc2d0> >, <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f9c03dcc300> >]
/////////////////
lambda_electrostatics_zero [<simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f9c03dcc390> >, <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7f9c03dcc420> >, <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7f9c03dcc450> >]
/////////////////
lambda_sterics_zero [<simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f9c03dcc330> >, <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f9c03dcc360> >, <simtk.openmm.openmm

In [4]:
reload(openmmtools.alchemy)
TS = openmmtools.states.ThermodynamicState(alchemical_system_in, temperature=300*unit.kelvin, pressure=1*unit.bar)
composable_states = [alchemical_state_zero, alchemical_state_one]
compound_state = openmmtools.states.CompoundThermodynamicState(
            thermodynamic_state=TS, composable_states=composable_states)

In [5]:
reload(openmmtools.alchemy)

compound_state._composable_states[0].lambda_sterics_zero = 0.0
compound_state._composable_states[0].lambda_electrostatics_zero = 1.0

compound_state._composable_states[1].lambda_sterics_one = 1.0
compound_state._composable_states[1].lambda_electrostatics_one = 1.0

sys = compound_state.get_system()
file = open('DEBUG_system.xml','w')
file.write(mm.XmlSerializer.serialize(sys))
file.close()

simulation = app.Simulation(pdb.topology, alchemical_system_in, integrator, platform, 
    properties)
compound_state.apply_to_context(simulation.context)

In [6]:
compound_state._composable_states[0].set_alchemical_variable('lambda', 1.0)
compound_state._composable_states[1].set_alchemical_variable('lambda', 1.0)

compound_state._composable_states[0].lambda_sterics_zero = openmmtools.alchemy.AlchemicalFunction('step_hm(lambda - 0.5) + 2*lambda * step_hm(0.5 - lambda)')
compound_state._composable_states[0].lambda_electrostatics_zero = openmmtools.alchemy.AlchemicalFunction('2*(lambda - 0.5) * step(lambda - 0.5)')

compound_state._composable_states[1].lambda_sterics_one = openmmtools.alchemy.AlchemicalFunction('step_hm(lambda - 0.5) + 2*lambda * step_hm(0.5 - lambda)')
compound_state._composable_states[1].lambda_electrostatics_one = openmmtools.alchemy.AlchemicalFunction('2*(lambda - 0.5) * step(lambda - 0.5)')

In [7]:
simulation.context.setPositions(pdb.positions)
'''
for param in simulation.context.getParameters():
    print(param)
    print(simulation.context.getParameter(param))
'''
print('Minimizing...')
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
#print('Equilibrating...')
#simulation.step(100)
#simulation.reporters.append(app.DCDReporter('trajectoryM.dcd', 1000))
#simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True, 
    #potentialEnergy=True, temperature=True, progress=True, remainingTime=True, 
    #speed=True, totalSteps=10000, separator='\t'))

# Collect data
nsteps = 2500 # number of steps per sample
niterations = 25 # number of samples to collect per alchemical state

lambdas = np.linspace(1.0, 0.0, 10)
lambdas_r =  np.linspace(0.0, 1.0, 10)# alchemical lambda schedule
print(lambdas)
nstates = len(lambdas)
u_kln = np.zeros([nstates,nstates,niterations], np.float64)
kT = unit.AVOGADRO_CONSTANT_NA * unit.BOLTZMANN_CONSTANT_kB * integrator.getTemperature()
for k in range(nstates):
    for iteration in range(niterations):
        print('state %5d iteration %5d / %5d' % (k, iteration, niterations))
        # Set alchemical state
        #context.setParameter('lambda', lambdas[k])
        compound_state._composable_states[0].set_alchemical_variable('lambda', lambdas[k])
        compound_state._composable_states[1].set_alchemical_variable('lambda', lambdas_r[k])
        compound_state.apply_to_context(simulation.context)
        print('lambda_sterics_zero',  'lambda_electrostatics_zero', 'lambda_sterics_one', 'lambda_electrostatics_one')
        print(simulation.context.getParameter('lambda_sterics_zero'), simulation.context.getParameter('lambda_electrostatics_zero')
            ,simulation.context.getParameter('lambda_sterics_one'), simulation.context.getParameter('lambda_electrostatics_one'))
        # Run some dynamics
        integrator.step(nsteps)
        # Compute energies at all alchemical states
        for l in range(nstates):
            compound_state._composable_states[0].set_alchemical_variable('lambda', lambdas[l])
            compound_state._composable_states[1].set_alchemical_variable('lambda', lambdas_r[l])
            compound_state.apply_to_context(simulation.context)
            u_kln[k,l,iteration] = simulation.context.getState(getEnergy=True).getPotentialEnergy() / kT

Minimizing...
[1.         0.88888889 0.77777778 0.66666667 0.55555556 0.44444444
 0.33333333 0.22222222 0.11111111 0.        ]
state     0 iteration     0 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 1.0 0.0 -0.0
state     0 iteration     1 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 1.0 0.0 -0.0
state     0 iteration     2 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 1.0 0.0 -0.0
state     0 iteration     3 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 1.0 0.0 -0.0
state     0 iteration     4 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 1.0 0.0 -0.0
state     0 iteration     5 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 1.0 0.0 -0.0
sta

state     2 iteration     1 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.5555555555555556 0.4444444444444444 -0.0
state     2 iteration     2 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.5555555555555556 0.4444444444444444 -0.0
state     2 iteration     3 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.5555555555555556 0.4444444444444444 -0.0
state     2 iteration     4 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.5555555555555556 0.4444444444444444 -0.0
state     2 iteration     5 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.5555555555555556 0.4444444444444444 -0.0
state     2 iteration     6 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_o

state     3 iteration    23 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.3333333333333335 0.6666666666666666 -0.0
state     3 iteration    24 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.3333333333333335 0.6666666666666666 -0.0
state     4 iteration     0 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.11111111111111116 0.8888888888888888 -0.0
state     4 iteration     1 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.11111111111111116 0.8888888888888888 -0.0
state     4 iteration     2 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
1.0 0.11111111111111116 0.8888888888888888 -0.0
state     4 iteration     3 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatic

state     5 iteration    20 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.8888888888888888 -0.0 1.0 0.11111111111111116
state     5 iteration    21 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.8888888888888888 -0.0 1.0 0.11111111111111116
state     5 iteration    22 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.8888888888888888 -0.0 1.0 0.11111111111111116
state     5 iteration    23 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.8888888888888888 -0.0 1.0 0.11111111111111116
state     5 iteration    24 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.8888888888888888 -0.0 1.0 0.11111111111111116
state     6 iteration     0 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostat

state     7 iteration    17 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.44444444444444464 -0.0 1.0 0.5555555555555554
state     7 iteration    18 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.44444444444444464 -0.0 1.0 0.5555555555555554
state     7 iteration    19 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.44444444444444464 -0.0 1.0 0.5555555555555554
state     7 iteration    20 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.44444444444444464 -0.0 1.0 0.5555555555555554
state     7 iteration    21 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.44444444444444464 -0.0 1.0 0.5555555555555554
state     7 iteration    22 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostat

state     9 iteration    17 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.0 -0.0 1.0 1.0
state     9 iteration    18 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.0 -0.0 1.0 1.0
state     9 iteration    19 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.0 -0.0 1.0 1.0
state     9 iteration    20 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.0 -0.0 1.0 1.0
state     9 iteration    21 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.0 -0.0 1.0 1.0
state     9 iteration    22 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.0 -0.0 1.0 1.0
state     9 iteration    23 /    25
lambda_sterics_zero lambda_electrostatics_zero lambda_sterics_one lambda_electrostatics_one
0.

In [8]:
# Estimate free energy of Lennard-Jones particle insertion
from pymbar import MBAR, timeseries
# Subsample data to extract uncorrelated equilibrium timeseries
N_k = np.zeros([nstates], np.int32) # number of uncorrelated samples
for k in range(nstates):
    [nequil, g, Neff_max] = timeseries.detectEquilibration(u_kln[k,k,:])
    indices = timeseries.subsampleCorrelatedData(u_kln[k,k,:], g=g)
    N_k[k] = len(indices)
    u_kln[k,:,0:N_k[k]] = u_kln[k,:,indices].T
# Compute free energy differences and statistical uncertainties
mbar = MBAR(u_kln, N_k)
[DeltaF_ij, dDeltaF_ij, Theta_ij] = mbar.getFreeEnergyDifferences()
print(DeltaF_ij)

[[  0.           7.25231285  13.59543946  16.44071605  17.28111292
   17.02439232  15.4688614   12.19697008   6.35230487  -0.84046884]
 [ -7.25231285   0.           6.34312661   9.1884032   10.02880007
    9.77207947   8.21654855   4.94465724  -0.90000798  -8.09278168]
 [-13.59543946  -6.34312661   0.           2.84527659   3.68567346
    3.42895286   1.87342194  -1.39846937  -7.24313459 -14.43590829]
 [-16.44071605  -9.1884032   -2.84527659   0.           0.84039687
    0.58367627  -0.97185465  -4.24374597 -10.08841118 -17.28118489]
 [-17.28111292 -10.02880007  -3.68567346  -0.84039687   0.
   -0.2567206   -1.81225152  -5.08414284 -10.92880805 -18.12158176]
 [-17.02439232  -9.77207947  -3.42895286  -0.58367627   0.2567206
    0.          -1.55553092  -4.82742223 -10.67208745 -17.86486115]
 [-15.4688614   -8.21654855  -1.87342194   0.97185465   1.81225152
    1.55553092   0.          -3.27189132  -9.11655653 -16.30933024]
 [-12.19697008  -4.94465724   1.39846937   4.24374597   5.084142

In [9]:
print(len(DeltaF_ij))
ans = 0.0
i=1
for j in range(0,len(DeltaF_ij)):
    #print(j,i)
    ans += DeltaF_ij[i,j]
    i+=1
    if i > 9:
        break
print(ans)

10
0.8404688373339901
